In [6]:
using CSV,DataFrames,Statistics,StatsBase,GLM,MLBase # ,BoxCoxTrans;

In [2]:
using Plots;
Plots.PlotMeasures;

In [7]:
plotlyjs();

In [17]:
using StatsPlots,GLMNet,StatsBase

In [21]:
using Random

In [9]:
# using CategoricalArrays
# this library is used to change the categorical arrays to numerical arrays

In [30]:
data

Row,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Int64,String7,Float64,Int64,Int64,String3,String15,String7,Float64,Float64,String15,Int64
1,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,27.7,never smoked,1
3,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
5,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
6,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
7,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
8,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
9,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,27.7,Unknown,1


In [10]:
data = CSV.read("E:/Datasets&Pictures for Coding/final-stroke-data-main-proj without missing vals.csv", DataFrame);

In [48]:
labelmaps = MLBase.labelmap(data.gender)
MLBase.labelencode(labelmaps,data.gender);

In [24]:
function perclass_splits(y,at) # y is the numeric column after transfrom #the label into numeric
    uids = unique(y)
    keepids = []
    for ui in uids
    curids = findall(y .==  ui)
    rowids = StatsBase.randsubseq(curids,at)
    push!(keepids,rowids)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [45]:
data_mat = Matrix(data)
train_ratio = 0.75
train_size = Int(round(size(data_mat,1)*train_ratio))
train_ind = randperm(size(data_mat,1))[1:train_size]
test_ind = setdiff(1:size(data_mat,1),train_ind)
train_data = data_mat[train_ind,:]
test_data = data_mat[test_ind,:]

1277×12 Matrix{Any}:
 31112  "Male"    80.0   0  1  …  "never smoked"     1
 60491  "Female"  78.0   0  0     "Unknown"          1
 12109  "Female"  81.0   1  0     "never smoked"     1
 58202  "Female"  50.0   1  0     "never smoked"     1
 70630  "Female"  71.0   0  0     "smokes"           1
 13861  "Female"  52.0   1  0  …  "never smoked"     1
 69160  "Male"    59.0   0  0     "formerly smoked"  1
 47269  "Male"    74.0   0  0     "formerly smoked"  1
 61960  "Male"    82.0   0  1     "smokes"           1
  7937  "Male"    60.0   1  0     "smokes"           1
 18587  "Female"  76.0   0  0  …  "Unknown"          1
  8752  "Female"  63.0   0  0     "never smoked"     1
 70676  "Female"  76.0   0  0     "formerly smoked"  1
     ⋮                                  ⋱  ⋮                            
 62425  "Female"   5.0   0  0  …  "Unknown"          0
 63997  "Male"    70.0   0  0     "Unknown"          0
 39935  "Female"  34.0   0  0     "never smoked"     0
  8203  "Male"    17.0   0

In [ ]:
println(propertynames(data))
println(names(data)) # what are the all columns of the dataset

In [28]:
size(data)
# this gives rows no  5110 and columns no 12

(5109, 12)

In [11]:
# it will determine type of each column in the dataframe
eltype.(eachcol(data))

12-element Vector{DataType}:
 Int64
 String7
 Float64
 Int64
 Int64
 String3
 String15
 String7
 Float64
 Float64
 String15
 Int64

In [12]:
new_data = data;

In [69]:
fm = @formula(stroke ~ gender+age+hypertension+heart_disease+
    ever_married+work_type+residence_type+avg_glucose_level+bmi+
    smoking_status )
log_model = glm(fm,new_data,Binomial(),ProbitLink(),L1Penalty())

LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

### Converting the categorical columns to a numerical data

* This part is required as when we want to calculate the correlation matrix or draw the correlation plot we need to give a matrix , so to make a matrix , we changed the values into numerical values.

In [31]:
first_col_new_data = recode(new_data[:,2], "Male" => 1, "Female" =>0 )
fifth_col_new_data = recode(new_data[:,6] ,"Yes" => 1, "No" => 0)
sixth_col_new_data = recode(new_data[:,7],"Private" => 1,
    "Self-employed" => 2,"Govt_job" => 3,"children" => 4, "Never_worked" => 5)
seventh_col_new_data = recode(new_data[:,8],"Urban" =>1,"Rural" => 0)
tenth_col_new_data = recode(new_data[:,11],"formerly smoked" => 1,
    "Unknown" => 2,"never smoked" => 3,"smokes" => 4);


In [32]:
# removing the columns those has categorical values
# they are gender(col 2),ever_married(col 6),work_type(col 7), residence_type(8), smoking_status(11).
new_data = select(new_data, Not([2,6,7,8,11]));

In [33]:
new_data = insertcols!(new_data, 2, :gender =>first_col_new_data)
new_data = insertcols!(new_data, 6, :ever_married => fifth_col_new_data)
new_data = insertcols!(new_data,7, :work_type =>sixth_col_new_data)
new_data = insertcols!(new_data, 8, :residence_type => seventh_col_new_data)
new_data = insertcols!(new_data,11, :smoking_status => tenth_col_new_data)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type
,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64
1,9046,1,67.0,0,1,1,1,1
2,51676,0,61.0,0,0,1,2,0
3,31112,1,80.0,0,1,1,1,0
4,60182,0,49.0,0,0,1,1,1
5,1665,0,79.0,1,0,1,2,0
6,56669,1,81.0,0,0,1,1,1
7,53882,1,74.0,1,1,1,1,0
8,10434,0,69.0,0,0,0,1,1
9,27419,0,59.0,0,0,1,1,0


In [34]:
# now we drop the id column, as that cols is not useful to our model building
new_data = select(new_data,Not([1]));

#### Below portion is for handelling missing values in the bmi column

#### the above parts are related to bmi missing values

In [26]:
# converting the dataframe into a matrix
df = Matrix(new_data);

In [27]:
# corrplot(df)
# this gives you the correlation plot between the predictors

## Finding the unique values for each predictor

In [35]:
# for the gender column
countmap(new_data[:,1])

Dict{Int64, Int64} with 2 entries:
  0 => 2994
  1 => 2115

In [36]:
countmap(new_data[:,end])

Dict{Int64, Int64} with 2 entries:
  0 => 4860
  1 => 249

In [30]:
# total true percentage
(249/size(new_data)[1])*100

4.873752201996477

## So, from here we can say that our data is not much balanced.

In [31]:
# counting all the elements in each column of the new data set
c1 = combine(groupby(new_data,2),nrow)
c2 = combine(groupby(new_data,3),nrow)
c3 = combine(groupby(new_data,4),nrow)
c4 = combine(groupby(new_data,5),nrow)
c5 = combine(groupby(new_data,6),nrow)
c6 = combine(groupby(new_data,7),nrow)
c7 = combine(groupby(new_data,8),nrow)
c8 = combine(groupby(new_data,9),nrow)
c9 = combine(groupby(new_data,10),nrow)
c10 = combine(groupby(new_data,11),nrow);

In [33]:
c10

,stroke,nrow
,Int64,Int64
1,1,249
2,0,4860


In [34]:
unique(new_data[:,9])

418-element Vector{Float64}:
 36.6
 27.7
 32.5
 34.4
 24.0
 29.0
 27.4
 22.8
 24.2
 29.7
 36.8
 27.3
 28.2
  ⋮
 45.8
 47.6
 14.0
 46.4
 46.9
 47.1
 13.3
 48.1
 51.7
 46.3
 54.1
 14.9

In [35]:
show(c1)

104×2 DataFrame
 Row │ age      nrow  
     │ Float64  Int64 
─────┼────────────────
   1 │   67.0      49
   2 │   61.0      76
   3 │   80.0      70
   4 │   49.0      79
   5 │   79.0      85
   6 │   81.0      60
   7 │   74.0      40
   8 │   69.0      54
   9 │   59.0      80
  10 │   78.0     102
  11 │   54.0      87
  ⋮  │    ⋮       ⋮
  95 │    0.72      5
  96 │    1.88      8
  97 │    1.24      8
  98 │    0.8       4
  99 │    0.4       2
 100 │    0.08      2
 101 │    1.48      6
 102 │    0.56      5
 103 │    0.48      3
 104 │    0.16      3
       83 rows omitted

In [36]:
show(c2)

2×2 DataFrame
 Row │ hypertension  nrow  
     │ Int64         Int64 
─────┼─────────────────────
   1 │            0   4611
   2 │            1    498

In [37]:
show(c3)

2×2 DataFrame
 Row │ heart_disease  nrow  
     │ Int64          Int64 
─────┼──────────────────────
   1 │             1    276
   2 │             0   4833

In [38]:
show(c4)

2×2 DataFrame
 Row │ ever_married  nrow  
     │ String3       Int64 
─────┼─────────────────────
   1 │ Yes            3353
   2 │ No             1756

In [39]:
show(c5)

5×2 DataFrame
 Row │ work_type      nrow  
     │ String15       Int64 
─────┼──────────────────────
   1 │ Private         2924
   2 │ Self-employed    819
   3 │ Govt_job         657
   4 │ children         687
   5 │ Never_worked      22

In [40]:
show(c6)

2×2 DataFrame
 Row │ Residence_type  nrow  
     │ String7         Int64 
─────┼───────────────────────
   1 │ Urban            2596
   2 │ Rural            2513

In [41]:
show(c7)

3978×2 DataFrame
  Row │ avg_glucose_level  nrow  
      │ Float64            Int64 
──────┼──────────────────────────
    1 │            228.69      1
    2 │            202.21      2
    3 │            105.92      1
    4 │            171.23      1
    5 │            174.12      1
    6 │            186.21      1
    7 │             70.09      1
    8 │             94.39      2
    9 │             76.15      1
   10 │             58.57      1
   11 │             80.43      2
  ⋮   │         ⋮            ⋮
 3969 │             76.89      1
 3970 │             95.05      1
 3971 │             95.02      1
 3972 │             79.15      1
 3973 │             76.62      1
 3974 │             83.94      1
 3975 │            125.2       1
 3976 │             82.99      1
 3977 │            166.29      1
 3978 │             85.28      1
                3957 rows omitted

In [42]:
show(c8)

418×2 DataFrame
 Row │ bmi      nrow  
     │ Float64  Int64 
─────┼────────────────
   1 │    36.6     14
   2 │    27.7    238
   3 │    32.5     21
   4 │    34.4     18
   5 │    24.0     28
   6 │    29.0     26
   7 │    27.4     22
   8 │    22.8     25
   9 │    24.2     29
  10 │    29.7     27
  11 │    36.8      8
  ⋮  │    ⋮       ⋮
 409 │    14.0      1
 410 │    46.4      1
 411 │    46.9      2
 412 │    47.1      1
 413 │    13.3      1
 414 │    48.1      1
 415 │    51.7      1
 416 │    46.3      1
 417 │    54.1      1
 418 │    14.9      1
      397 rows omitted

In [43]:
show(c9)

4×2 DataFrame
 Row │ smoking_status   nrow  
     │ String15         Int64 
─────┼────────────────────────
   1 │ formerly smoked    884
   2 │ never smoked      1892
   3 │ smokes             789
   4 │ Unknown           1544

In [44]:
show(c10)

2×2 DataFrame
 Row │ stroke  nrow  
     │ Int64   Int64 
─────┼───────────────
   1 │      1    249
   2 │      0   4860

So, by looking at the stroke column, we can say that, this dataset is quite inbalanced as here true stroke values only 249 whereas the no stroke values is 4860.

#  Visualization of the predictors

In [45]:
using StatsPlots

In [46]:
density(new_data.bmi,bandwidth = 2,lw=3,c=4) 
plot!(legend =false)
histogram!(new_data.bmi,label =false,normalize=true)
# It make a line plot of a kernel density estimate of x,

In [47]:
boxplot(new_data.bmi)

In [48]:
histogram(new_data[:,"age"],normalize = true,label = false,c = :grey)
plot!(fit(Normal,new_data[:,2]))

In [49]:
using StatsPlots

In [50]:
# age is 2nd column
p1 = histogram(new_data[:,"age"],normalize = true,label = false,c = :grey)

p2 = boxplot(new_data[:,"age"],label = "age")

# from the boxplot we can say that, there is no outliers in age column

plot(p1,p2)


In [51]:
fit(Normal,new_data[:,2])

Normal{Float64}(μ=43.2299862986886, σ=22.611362087649784)

In [52]:
import Distributions

In [53]:
# see the effect of avg_glucose_level predictors 
p = boxplot((new_data[:,"avg_glucose_level"]),label = "average glucose level",fg_legend =false,legend = :topright)
p1 = histogram((new_data[:,"avg_glucose_level"]),normalize = true,c = :grey,
    xlabel ="average glucose level" ,ylabel = "density",label = :none)
plot(p,p1)


In [54]:
histogram((new_data[:,"avg_glucose_level"]),normalize = true,c = :grey,
    xlabel ="average glucose level" ,ylabel = "density",label = :none)
plot!(fit(Normal,new_data[:,:avg_glucose_level]))


The histograms are looks like it is a bimodal distribution, so we don't consider the outside points as outliers.

In [55]:
p = boxplot(new_data[:,"bmi"],label = "bmi",fg_legend =false)
p1 = histogram(new_data[:,"bmi"],normalize = true,c = :grey, xlabel =:bmi ,ylabel = "density",label = :none)
plot(p,p1)


In [56]:
histogram(new_data[:,"bmi"],normalize = true,c = :grey, xlabel =:bmi ,ylabel = "density",label = :none)
plot!(fit(Normal,new_data[:,:bmi]))

By looking at the histogram, we can see that, the bmi values are positively skewed, so we apply a log transformation.

In [57]:
#transform the bmi column by transform with log function
p2 = boxplot(log.(new_data[:,"bmi"]),label = :none)
p3 = histogram(log.(new_data[:,"bmi"]),label = "transformed bmi",
    normalize = true)
plot(p2,p3)


# relation with Predictors and Response variable

In [58]:
boxplot(new_data.stroke,new_data.age,label = :none)

In [59]:
# see the relation with age predictor and response stroke
scatter(new_data[:,"age"],new_data[:,end],
    ylabel = "stroke",xlabel ="age",label = :none )
#savefig("logmodel 5")

From this plot, we can say that, as age increase the patient have a stroke with more probability.

In [60]:
scatter(new_data[:,8],new_data[:,end],
    ylabel = "stroke",xlabel = "glucose level",label = :none)
#savefig("logmodel 6")

From this plot, we can say that average glucose level also have good effect to the response stroke.

In [61]:
scatter(log.(new_data[:,9]),new_data[:,end],
    ylabel = "stroke",xlabel = "transformed bmi",label = :none)


From this plot, we can say said that, the updated bmi values may have an effect to the response stroke values.

In [62]:
fit(Normal,new_data[:,8])

Normal{Float64}(μ=106.14039929536098, σ=45.280571563580956)

In [63]:
using Distributions

In [64]:
fit(Cauchy,new_data[:,8])

Cauchy{Float64}(μ=91.88, σ=18.425000000000004)

# Changing the values of the predictors based on above diagrams

In [65]:
new_data[:,"bmi"] = log.(new_data[:,"bmi"]);

#### Since it is a classification problem, don't need to use Linear model Fit a linear model to the dataset with all predictors.

#### By looking at the p- values for all the estimated coefficients we can say, gender,work_type,residence_type,smoking_status are not much significant

The linear/logistic regression model returns the probability for each row. We need to convert the probabilities ti binary outcomes(0 and 1) as our problem is on binary classification(stroke or not). For that,we need to decide a cut-off. We can consider the empirical proportion of the strokes in the data as the cut-off. 

For example, in this problem, it will be the number of rows with stroke =1 divided by the number of rows in the data. 

Consider that value as cut-off and check the performence of the linear/logistic model.

In [66]:
countmap(new_data.stroke)

Dict{Int64, Int64} with 2 entries:
  0 => 4860
  1 => 249

In [67]:
fm = @formula(stroke ~ gender)
log_model = glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       z  Pr(>|z|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   -1.67371    0.0393786  -42.50    <1e-99  -1.75089    -1.59653
gender: Male   0.039081   0.0602784    0.65    0.5168  -0.0790626   0.157225
────────────────────────────────────────────────────────────────────────────

In [68]:
coef(log_model)

2-element Vector{Float64}:
 -1.6737060875922356
  0.03908097332865651

###  Use logistic classifier model with all the predictors

In [70]:
bic_all = bic(log_model)

2006.9306697603308

In [71]:
aic(log_model)

1993.8531518216691

## Drop gender

In [72]:
fm = @formula(stroke ~ age+hypertension+heart_disease+
    ever_married+work_type+residence_type+avg_glucose_level+bmi+
    smoking_status )
log_model_1 = glm(fm,new_data,Binomial(),ProbitLink())

LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [73]:
bic_one = bic(log_model_1)

LoadError: UndefVarError: log_model_1 not defined

In [74]:
aic(log_model_1)

LoadError: UndefVarError: log_model_1 not defined

## convert categorical to numerical without using in built function

In [75]:
select(DataFrame(gender=data[!,:gender]),
[:gender => ByRow(isequal(v)) => Symbol(v)
        for v in unique(DataFrame(gender= data[!,:gender]).gender)])

,Male,Female,Other
,Bool,Bool,Bool
1,1,0,0
2,0,1,0
3,1,0,0
4,0,1,0
5,0,1,0
6,1,0,0
7,1,0,0
8,0,1,0
9,0,1,0


## drop bmi

In [76]:
fm = @formula(stroke ~ age+hypertension+heart_disease+
    ever_married+work_type+residence_type+avg_glucose_level+
    smoking_status )
log_model_2 = glm(fm,new_data,Binomial(),ProbitLink())

LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [77]:
bic_2 = bic(log_model_2)

LoadError: UndefVarError: log_model_2 not defined

In [78]:
aic(log_model_2)

LoadError: UndefVarError: log_model_2 not defined

## drop smoking status

In [79]:
fm = @formula(stroke ~ age+hypertension+heart_disease+
    ever_married+work_type+residence_type+avg_glucose_level)
log_model_3= glm(fm,new_data,Binomial(),ProbitLink())

LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [80]:
bic_3 = bic(log_model_3)

LoadError: UndefVarError: log_model_3 not defined

In [81]:
aic(log_model_3)

LoadError: UndefVarError: log_model_3 not defined

## drop residence type

In [82]:
fm = @formula(stroke ~ age+hypertension+heart_disease+
    ever_married+work_type+avg_glucose_level)
log_model_4= glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + hypertension + heart_disease + ever_married + work_type + avg_glucose_level

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────────────
                                Coef.   Std. Error       z  Pr(>|z|)      Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)               -3.89917     0.203884     -19.12    <1e-80   -4.29878      -3.49957
age                        0.0353287   0.00268934    13.14    <1e-38    0.0300577     0.0405997
hypertension               0.215126    0.086441       2.49    0.0128    0.0457048     0.384547
heart_disease              0.175855    0.103416       1.70    0.0890   -0.0268

In [83]:
bic_4 = bic(log_model_4)

1665.55199668121

In [84]:
aic(log_model_4)

1600.164406987902

## drop work type

In [85]:
fm = @formula(stroke ~ age+hypertension+heart_disease+
    ever_married+avg_glucose_level)
log_model_5= glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + hypertension + heart_disease + ever_married + avg_glucose_level

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                         Coef.   Std. Error       z  Pr(>|z|)     Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.65612     0.165615     -22.08    <1e-99  -3.98072      -3.33152
age                 0.032402    0.00243201    13.32    <1e-39   0.0276353     0.0371686
hypertension        0.208492    0.0860821      2.42    0.0154   0.0397741     0.37721
heart_disease       0.189665    0.102944       1.84    0.0654  -0.0121014     0.391432
ever_married: Yes  -0.125045    0.106789      -1.17    

In [86]:
bic_5 = bic(log_model_5)

1639.5276795285495

In [87]:
aic(log_model_5)

1600.2951257125646

## drop ever married

In [88]:
fm = @formula(stroke ~ age+hypertension+heart_disease+avg_glucose_level)
log_model_6= glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + hypertension + heart_disease + avg_glucose_level

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                         Coef.  Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.72084     0.160497    -23.18    <1e-99  -4.03541      -3.40628
age                 0.0317519   0.00239845   13.24    <1e-39   0.027051      0.0364528
hypertension        0.210217    0.0859421     2.45    0.0144   0.0417732     0.37866
heart_disease       0.196258    0.10271       1.91    0.0560  -0.00504921    0.397565
avg_glucose_level   0.00206046  0.00060919    3.38    0.0007   0.000866468

In [89]:
bic_6 = bic(log_model_6)

1632.3235124268724

In [90]:
aic(log_model_6)

1599.6297175802183

## drop heart disease

In [91]:
fm = @formula(stroke ~ age+hypertension+avg_glucose_level)
log_model_7= glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + hypertension + avg_glucose_level

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                         Coef.  Std. Error       z  Pr(>|z|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.7696      0.159358    -23.65    <1e-99  -4.08193     -3.45726
age                 0.0326408   0.0023585    13.84    <1e-42   0.0280182    0.0372633
hypertension        0.21124     0.0858044     2.46    0.0138   0.0430665    0.379414
avg_glucose_level   0.00221075  0.00060389    3.66    0.0003   0.00102715   0.00339436
──────────────────────────────────────────────────────────────────────────────────────

In [92]:
bic_7 = bic(log_model_7)

1627.362817754075

In [93]:
aic(log_model_7)

1601.2077818767518

In [94]:
x = Vector(3:1:10)
y = [bic_7,bic_6,bic_5,bic_4,bic_3,bic_2,bic_one,bic_all]

LoadError: UndefVarError: bic_3 not defined

In [95]:
plot(x,y,legend = false,
    xlabel = "no of predictors",
    ylabel = "BIC to corresponding model")
# savefig("logmodel 8")

LoadError: UndefVarError: y not defined

### Let's take forward variable  selection process

In [96]:
fm = @formula(stroke ~ 1) # null model
forward_log_model= glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1

Coefficients:
─────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error       z  Pr(>|z|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  -1.65722   0.0298105  -55.59    <1e-99   -1.71565   -1.59879
─────────────────────────────────────────────────────────────────────────

In [97]:
aic(forward_log_model)

1992.2726779680813

In [98]:
fm = @formula(stroke ~ gender)
forward_log_model_1 = glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       z  Pr(>|z|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   -1.67371    0.0393786  -42.50    <1e-99  -1.75089    -1.59653
gender: Male   0.039081   0.0602784    0.65    0.5168  -0.0790626   0.157225
────────────────────────────────────────────────────────────────────────────

In [99]:
aic(forward_log_model_1) ## gender has no impact

1993.8531518216691

In [100]:
fm = @formula(stroke ~ gender + age)
forward_log_model_2= glm(fm,new_data,Binomial(),ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender + age

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error       z  Pr(>|z|)   Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   -3.61741    0.15116     -23.93    <1e-99  -3.91368    -3.32114
gender: Male   0.0394655  0.0686556     0.57    0.5654  -0.095097    0.174028
age            0.0348767  0.00229295   15.21    <1e-51   0.0303826   0.0393708
──────────────────────────────────────────────────────────────────────────────

In [101]:
aic(forward_log_model_2) # age has more impact

1620.5539286318885

In [102]:
fm = @formula(stroke ~ gender + age + hypertension)
forward_log_model_3= glm(fm,new_data,Binomial(),ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender + age + hypertension

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error       z  Pr(>|z|)   Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   -3.59512    0.151748    -23.69    <1e-99  -3.89255    -3.2977
gender: Male   0.038831   0.0688415     0.56    0.5727  -0.0960959   0.173758
age            0.0337408  0.00233063   14.48    <1e-46   0.0291728   0.0383087
hypertension   0.249731   0.0846765     2.95    0.0032   0.0837677   0.415693
──────────────────────────────────────────────────────────────────────────────

In [103]:
aic(forward_log_model_3) # hypertension also has some impact

1614.0883338724675

In [104]:
fm = @formula(stroke ~ gender + age + hypertension + heart_disease)
forward_log_model_4= glm(fm,new_data,Binomial(),ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender + age + hypertension + heart_disease

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error       z  Pr(>|z|)   Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)    -3.54374    0.152355    -23.26    <1e-99  -3.84235    -3.24513
gender: Male    0.0180881  0.0695996     0.26    0.7950  -0.118325    0.154501
age             0.0325784  0.00237596   13.71    <1e-42   0.0279216   0.0372352
hypertension    0.24539    0.0849213     2.89    0.0039   0.0789473   0.411833
heart_disease   0.238132   0.102474      2.32    0.0201   0.0372871   0.438976
─────────────────────────────────────────────────────────

In [105]:
aic(forward_log_model_4) # heart disease also has some impact

1610.7784686427199

In [106]:
fm = @formula(stroke ~gender +  age + hypertension + heart_disease+ever_married)
forward_log_model_5= glm(fm,new_data,Binomial(),ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender + age + hypertension + heart_disease + ever_married

Coefficients:
───────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error       z  Pr(>|z|)   Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.4876     0.159365    -21.88    <1e-99  -3.79995    -3.17525
gender: Male        0.02142    0.0696716     0.31    0.7585  -0.115134    0.157974
age                 0.0331204  0.00241653   13.71    <1e-42   0.0283841   0.0378568
hypertension        0.244503   0.0850311     2.88    0.0040   0.0778446   0.411161
heart_disease       0.23306    0.102651      2.27    0.0232   0.0318674   0.434252
ever_marri

In [107]:
aic(forward_log_model_5) # ever married no impact

1611.8694194856798

In [108]:
fm = @formula(stroke ~ gender + age + hypertension + heart_disease+ever_married + work_type)
forward_log_model_6= glm(fm,new_data,Binomial(),ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + gender + age + hypertension + heart_disease + ever_married + work_type

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────────
                               Coef.  Std. Error       z  Pr(>|z|)    Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────────────────────
(Intercept)               -3.73721    0.198014    -18.87    <1e-78   -4.12531    -3.34911
gender: Male               0.0128527  0.069937      0.18    0.8542   -0.124221    0.149927
age                        0.036112   0.00266817   13.53    <1e-41    0.0308825   0.0413415
hypertension               0.25055    0.0854104     2.93    0.0034    0.0831492   0.417952
heart_disease            

In [109]:
aic(forward_log_model_6) ## work type also has no impact

1610.9574489149672

In [110]:
fm = @formula(stroke ~ gender + age + hypertension + heart_disease+ever_married + work_type+residence_type)
forward_log_model_7= glm(fm,new_data,Binomial(),ProbitLink())


LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [111]:
aic(forward_log_model_7) # residence type has no impact

LoadError: UndefVarError: forward_log_model_7 not defined

In [112]:
fm = @formula(stroke ~ gender + age + hypertension + heart_disease+ever_married + work_type+
    residence_type+avg_glucose_level)
forward_log_model_8= glm(fm,new_data,Binomial(),ProbitLink())


LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [113]:
aic(forward_log_model_8) #avg glucose levl has impact

LoadError: UndefVarError: forward_log_model_8 not defined

In [114]:
fm = @formula(stroke ~ gender + age + hypertension + heart_disease+ever_married + work_type+
    residence_type+avg_glucose_level+bmi)
forward_log_model_9= glm(fm,new_data,Binomial(),ProbitLink())


LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [115]:
aic(forward_log_model_9) # bmi has no impact

LoadError: UndefVarError: forward_log_model_9 not defined

In [116]:
fm = @formula(stroke ~ gender + age + hypertension + heart_disease+ever_married + work_type+
    residence_type+avg_glucose_level+bmi+smoking_status)
forward_log_model_10= glm(fm,new_data,Binomial(),ProbitLink())


LoadError: ArgumentError: There isn't a variable called 'residence_type' in your data; the nearest names appear to be: Residence_type

In [117]:
aic(forward_log_model_10) # smoking status has  no impact 

LoadError: UndefVarError: forward_log_model_10 not defined

## hybrid selection method

In [118]:
fm = @formula(stroke ~ 1) # null model
hybrid_log_model= glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1

Coefficients:
─────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error       z  Pr(>|z|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  -1.65722   0.0298105  -55.59    <1e-99   -1.71565   -1.59879
─────────────────────────────────────────────────────────────────────────

In [119]:
fm = @formula(stroke ~ age) # null model
forward_log_model= aic(glm(fm,new_data,Binomial(),ProbitLink()))

1618.8836133265052

In [120]:
fm = @formula(stroke ~age + hypertension + heart_disease+avg_glucose_level+bmi)
forward_log_model_6= glm(fm,new_data,Binomial(),ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + hypertension + heart_disease + avg_glucose_level + bmi

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                         Coef.   Std. Error      z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.64357     0.595356     -6.12    <1e-09  -4.81044      -2.47669
age                 0.0317113   0.00240729   13.17    <1e-38   0.0269931     0.0364295
hypertension        0.211148    0.0862701     2.45    0.0144   0.0420612     0.380234
heart_disease       0.195959    0.102762      1.91    0.0565  -0.00545036    0.397368
avg_glucose_level   0.00207771  0.000620559   3.35    0.0008   0.00

In [121]:
fm = @formula(stroke ~ age+ avg_glucose_level + heart_disease + hypertension)
glm(fm,new_data,Binomial(),ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + avg_glucose_level + heart_disease + hypertension

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                         Coef.  Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.72084     0.160497    -23.18    <1e-99  -4.03541      -3.40628
age                 0.0317519   0.00239845   13.24    <1e-39   0.027051      0.0364528
avg_glucose_level   0.00206046  0.00060919    3.38    0.0007   0.000866468   0.00325445
heart_disease       0.196258    0.10271       1.91    0.0560  -0.00504921    0.397565
hypertension        0.210217    0.0859421     2.45    0.0144   0.041773

In [122]:
MixedModelsExtras.vif(log_model_7)
# this gives vif for each predictor

LoadError: UndefVarError: MixedModelsExtras not defined

In [123]:
aic(log_model) # Akaike Information criterion

1993.8531518216691

In [124]:
bic(log_model)
# Bayesian Information Criterion, defined as -2 \log L + k \log n, with L the
# likelihood of the model, k its number of consumed degrees of freedom and n the number of observations

2006.9306697603308

In [125]:
cut_off = (249/nrow(new_data))*100

4.873752201996477

In [126]:
fit_pred = predict(log_model_7,select(new_data,Not([11])));
# predicting on the whole dataset


# with log model 6

In [127]:
fit_pred_6 = predict(log_model_6,select(new_data,Not([11])))
fit_predict_6 = Vector{Int64}(fit_pred_6 .> cut_off)
fit_predict_6 = Vector{Int64}(fit_predict_6);

In [128]:
v1 = MLBase.roc(fit_predict_6,new_data[:,end])

ROCNums{Int64}
  p = 0
  n = 5109
  tp = 0
  tn = 4860
  fp = 249
  fn = 0


In [129]:
conf_mat = [v1.tn v1.fp;v1.fn v1.tp]

2×2 Matrix{Int64}:
 4860  249
    0    0

In [130]:
((conf_mat[1,1]+ conf_mat[2,2])/sum(conf_mat))*100

95.12624779800353

# with log model 7

In [131]:
fit_predict = Vector{Int64}(fit_pred .> cut_off);
fit_predict = Vector{Int64}(fit_predict);

In [132]:
v = MLBase.roc(fit_predict,new_data[:,end])

ROCNums{Int64}
  p = 0
  n = 5109
  tp = 0
  tn = 4860
  fp = 249
  fn = 0


In [133]:
countmap(new_data[:,end])

Dict{Int64, Int64} with 2 entries:
  0 => 4860
  1 => 249

In [134]:
(249/size(new_data)[1])*100

4.873752201996477

In [135]:
# construct the confusion matrix
confu_matrix = [v.tp v.fp; v.fn v.tn]

# out of 249, model predicts true stroke as true in 203 cases.

2×2 Matrix{Int64}:
 0   249
 0  4860

In [136]:
# calculating the accuracy
acc = (confu_matrix[2,2] + confu_matrix[1,1])/sum(confu_matrix)

0.9512624779800353

In [137]:
## sensitivity tells us what fraction of heart stroke does the model find
## that is how much classifier finds of what there actually is to find
sensitivity = Recall = (confu_matrix[2,2])/(confu_matrix[2,1] + confu_matrix[2,2])

1.0

In [138]:
## specificity tells us what fraction of non-stroke does the model find
specificity = confu_matrix[1,1]/(confu_matrix[1,1]+confu_matrix[1,2])

0.0

In [139]:
## If our model says that it is a stroke person then precision tells us that with this much probability
## the person is truely a  stroke  person.
Precision = confu_matrix[2,2]/(confu_matrix[2,2] + confu_matrix[1,2])

0.9512624779800353

In [140]:
## F1 score is the harmonic mean of precision and recall
## F1 score is closer to 1 tells us that the model is good
F1_score = (2*Precision*Recall)/(Precision+Recall)

0.9750225699668974

#### After splitting the dataset and run the model for the one times.

In [141]:
cut_off=0.5

0.5

In [142]:
names(new_data)

11-element Vector{String}:
 "gender"
 "age"
 "hypertension"
 "heart_disease"
 "ever_married"
 "work_type"
 "Residence_type"
 "avg_glucose_level"
 "bmi"
 "smoking_status"
 "stroke"

In [143]:
using PyCall

In [144]:
@pyimport statsmodels.stats.outliers_influence as vif

In [145]:
[vif.variance_inflation_factor(Array(new_data),i) for i in [2,3,4,8]]

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\hp\.julia\packages\PyCall\twYvK\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'AttributeError'>
AttributeError("'list' object has no attribute 'shape'")
  File "C:\Users\hp\.julia\conda\3\lib\site-packages\statsmodels\stats\outliers_influence.py", line 189, in variance_inflation_factor
    k_vars = exog.shape[1]


In [146]:
train_data1,test_data1 = TrainTestSplit(new_data,0.7)
fm = @formula(stroke ~ age + hypertension + heart_disease+avg_glucose_level)
log_model1 = glm(fm,train_data1,Binomial(),ProbitLink())
fit_predict1 = predict(log_model1,select(train_data1,Not([11])))
fit_predict1 = Vector{Int64}(fit_predict1 .> cut_off)
v1 = roc(fit_predict1,train_data1[:,end])
conf_mat1 = [v1.tn v1.fp; v1.fn v1.tp]
acc1 = ((conf_mat1[2,2] + conf_mat1[1,1])/sum(conf_mat1))*100 # this is training accuracy

94.99020431010355

In [147]:
log_model1

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

stroke ~ 1 + age + hypertension + heart_disease + avg_glucose_level

Coefficients:
────────────────────────────────────────────────────────────────────────────────────────
                         Coef.   Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -3.67489     0.187868     -19.56    <1e-84  -4.0431       -3.30667
age                 0.0309031   0.00280963    11.00    <1e-27   0.0253963     0.0364098
hypertension        0.216007    0.101218       2.13    0.0328   0.0176235     0.41439
heart_disease       0.257111    0.120631       2.13    0.0331   0.0206794     0.493543
avg_glucose_level   0.00213767  0.000722944    2.96    0.0031   0.0

In [148]:
conf_mat1

2×2 Matrix{Int64}:
 3394  179
    0    0

In [149]:
countmap(train_data1[:,end])

Dict{Int64, Int64} with 2 entries:
  0 => 3394
  1 => 179

In [150]:
size(train_data1)

(3573, 11)

In [151]:
log_model1 = glm(fm,test_data1,Binomial(),ProbitLink())
fit_predict1 = predict(log_model1,select(test_data1,Not([11])))
fit_predict1 = Vector{Int64}(fit_predict1 .> cut_off)
v1 = roc(fit_predict1,test_data1[:,end])
conf_mat1 = [v1.tn v1.fp; v1.fn v1.tp]
acc1 = ((conf_mat1[2,2]+conf_mat1[1,1])/sum(conf_mat1))*100 # calculate the test accuracy

95.44270833333334

In [152]:
sensitivity = Recall = (conf_mat1[2,2])/(conf_mat1[2,1] + conf_mat1[2,2])

NaN

In [153]:
specificity = conf_mat1[1,1]/(conf_mat1[1,1]+conf_mat1[1,2])

0.9544270833333334

In [154]:
Precision = conf_mat1[2,2]/(conf_mat1[2,2] + conf_mat1[1,2])

0.0

In [155]:
F1_score = (2*Precision*Recall)/(Precision+Recall)

NaN

* remove the predictors gender,heart disease,ever_married,residence_type,smoking_status

In [156]:
rep = 500
train_conf_mat = []
test_conf_mat = []
for i in 1:rep
    train_data,test_data = TrainTestSplit(new_data,0.7)
    fm = @formula(stroke ~ age+hypertension+avg_glucose_level+heart_disease )
    log_model = glm(fm,train_data,Binomial(),ProbitLink())
    fit_predict = predict(log_model,select(train_data,Not([11])))
    fit_predict = Vector{Int64}(fit_predict .> cut_off)
    v = MLBase.roc(fit_predict,train_data[:,end])
    conf_matrix = [v.tn v.fp; v.fn v.tp]
    push!(train_conf_mat,conf_matrix)
    
    fit_predict_test = predict(log_model,select(test_data,Not([11])))
    fit_predict_test = Vector{Int64}(fit_predict_test .> cut_off)
    v = MLBase.roc(fit_predict_test,test_data[:,end])
    conf_matrix = [v.tn v.fp; v.fn v.tp]
    push!(test_conf_mat,conf_matrix)
end

train_acc_overall = []
test_acc_overall = []
train_acc_correct = []
test_acc_correct  = []
for i in 1:rep
    mat = train_conf_mat[i]
    push!(train_acc_correct,mat[2,2]/(mat[2,1] + mat[2,2]))
    push!(train_acc_overall,(mat[1,1] + mat[2,2])/sum(mat))
end


for i in 1:rep
    mat = test_conf_mat[i]
    push!(test_acc_correct,mat[2,2]/(mat[2,1] + mat[2,2]))
    push!(test_acc_overall,(mat[1,1] + mat[2,2])/sum(mat))
end


In [157]:
## train acc overall is calculating the accuracy
boxplot(train_acc_overall,label = "train")
boxplot!(test_acc_overall,label = "test",
    title = "Overall accuracy",legend = :topleft)
#savefig("logmodel 9")

In [158]:
## train acc correct is finds how much the
# classifier actually finds what are stroke
boxplot(train_acc_correct,label = "train")
boxplot!(test_acc_correct,label = "test",
    title = "True positive rate",legend = :topleft)
# savefig("logmodel 10")

LoadError: ArgumentError: quantiles are undefined in presence of NaNs or missing values

In [159]:
## It approximates the train acc correct and 
# test acc correct by a kernel density estimator
density(train_acc_correct,label = "train") 
density!(test_acc_correct,label = "test",
    title = "Probability of Correct")
#savefig("logmodel 11")

LoadError: ArgumentError: quantiles are undefined in presence of NaNs or missing values

In [160]:
## It approximates the train acc overall and test acc overall by a kernel density function
density(train_acc_overall,label = "train") 
density!(test_acc_overall,label = "test",
    title = "Overall Accuracy")
# savefig("logmodel 12")

### Tree classification model, Random Forest, Bagging, Boosting,Ensemble learning.

In [161]:
using DecisionTree

In [162]:
train,test = TrainTestSplit(new_data,0.7) # that means 70% data is used for building the model
train_pred = train[:,1:end-1]
train_response = train[:,end];
test_pred = test[:,1:end-1]
test_response = test[:,end];

In [163]:
training_pred = Matrix(train_pred); 
model = DecisionTreeClassifier(max_depth = 7,
    impurity_importance = true,n_subfeatures = 10,min_samples_leaf = 5,
    min_samples_split = 6,pruning_purity_threshold = 4)
# impurity importance is helps us to calculate the feature importance 
DecisionTree.fit!(model,training_pred,train_response)

DecisionTreeClassifier
max_depth:                7
min_samples_leaf:         5
min_samples_split:        6
min_purity_increase:      0.0
pruning_purity_threshold: 4.0
n_subfeatures:            10
classes:                  [0, 1]
root:                     Decision Tree
Leaves: 50
Depth:  7

In [164]:
DecisionTree.get_classes(model)

2-element Vector{Int64}:
 0
 1

In [165]:
DecisionTree.impurity_importance(model,normalize = true)

10-element Vector{Float64}:
 0.0
 0.5722633608142056
 0.014142998866783987
 0.01344400802884628
 0.017391951881995372
 0.03140704281285101
 0.011165645668757622
 0.1759285067581873
 0.13662210361341487
 0.02763438155495783

In [166]:
sort(DecisionTree.impurity_importance(model,normalize = true),rev = true)

10-element Vector{Float64}:
 0.5722633608142056
 0.1759285067581873
 0.13662210361341487
 0.03140704281285101
 0.02763438155495783
 0.017391951881995372
 0.014142998866783987
 0.01344400802884628
 0.011165645668757622
 0.0

Feature importance is computed as follows:

    •  Single tree: For each feature, the associated importance is the
       sum, over all splits based on that feature, of the impurity
       decreases for that split (the node impurity minus the sum of the
       child impurities) divided by the total number of training
       observations. When normalize was true, the feature importances
       were normalized by the sum of feature importances.

  More explicitly, the impurity decrease for node i is:

  Δimpurityᵢ = nᵢ × lossᵢ - nₗ × lossₗ - nᵣ × lossᵣ

  Where n is the number of observations, loss is entropy, gini index or other
  measures of impurity, index i denotes the quantity of node i, index l
  denotes the quantity of left child node, and index r denotes the quantity of
  right child node.

    •  Forests: The importance for a given feature is the average over
       trees in the forest of the normalized tree importances for that
       feature.

    •  AdaBoost models: The feature importance is as same as
       split_importance.

  For forests and adaboost models, feature importance is normalized before
  averaging over trees, so the keyword argument normalize is useless. Whether
  to normalize or not is controversial, but the current implementation is
  identical to scikitlearn's RandomForestClassifier, RandomForestRegressor,
  and AdaBoostClassifier, which is different from feature importances
  described in G. Louppe, “Understanding Random Forests: From Theory to
  Practice”, PhD Thesis, U. of Liege, 2014. (https://arxiv.org/abs/1407.7502).
  See this PR (https://github.com/scikit-learn/scikit-learn/issues/19972) for
  a detailed discussion.

In [167]:
# print_tree(model)

In [168]:
prediction_train = DecisionTree.predict(model,Matrix(train_pred))
acc_train = (mean(prediction_train .== train_response))*100

95.71150097465888

In [169]:
prediction_test = DecisionTree.predict(model,Matrix(test_pred));
acc_test = mean(prediction_test .== test_response)*100 
# test accuracy of the  decision tree model

93.67588932806325

In [170]:
train_conf_mat = (DecisionTree.confusion_matrix(train_response,
        prediction_train)).matrix

2×2 Matrix{Int64}:
 3432  3
  151  5

In [171]:
train_precision = (train_conf_mat[2,2]/(train_conf_mat[2,2] + train_conf_mat[1,2]))*100

62.5

In [172]:
countmap(train_response)

Dict{Int64, Int64} with 2 entries:
  0 => 3435
  1 => 156

In [173]:
test_conf_mat = DecisionTree.confusion_matrix(test_response,
    prediction_test).matrix

2×2 Matrix{Int64}:
 1422  3
   93  0

In [174]:
test_precision = (test_conf_mat[2,2]/(test_conf_mat[2,2] + test_conf_mat[1,2]))*100

0.0

In [175]:
countmap(test_response)

Dict{Int64, Int64} with 2 entries:
  0 => 1425
  1 => 93

In [176]:
DecisionTree.predict_proba(model,Matrix(test_pred));

## run the decision tree model 100 times

In [177]:
rep = 500
conf_mat_train = []
conf_mat_test = []
train_acc_overall = []
train_acc_correct = []
test_acc_overall = []
test_acc_correct = []
for i in 1:rep
    train,test = TrainTestSplit(new_data,0.7)
    train_pred = train[:,1:end-1]
    train_response = train[:,end];
    test_pred = test[:,1:end-1]
    test_response = test[:,end];
    training_pred = Matrix(train_pred);
    model = DecisionTreeClassifier(max_depth = 7,impurity_importance = true,n_subfeatures = 10)
    DecisionTree.fit!(model,training_pred,train_response)
    prediction_train = DecisionTree.predict(model,Matrix(train_pred))
    push!(conf_mat_train,DecisionTree.confusion_matrix(train_response,prediction_train).matrix)
    push!(train_acc_overall,mean(prediction_train .== train_response))
    
    
    prediction_test = DecisionTree.predict(model,Matrix(test_pred));
    push!(conf_mat_test,DecisionTree.confusion_matrix(test_response,prediction_test).matrix)
    push!(test_acc_overall,mean(prediction_test .== test_response))
end

for i in 1:rep
    mat = conf_mat_train[i]
    push!(train_acc_correct,mat[2,2]/(mat[2,1] + mat[2,2]))
end

for i in 1:rep
    mat = conf_mat_test[i]
    push!(test_acc_correct,mat[2,2]/(mat[2,1] + mat[2,2]))
end

In [178]:
p = boxplot(train_acc_correct,label = "train")
p1 = boxplot(test_acc_correct,label = "test",
    title = "True positive rate",legend = :topleft)
plot(p,p1)
#savefig("decision tree 1")

In [179]:
density(train_acc_correct,label = "train",trim =true)
density!(test_acc_correct,label= "test",title = "Probability of correct",legend = :topleft,trim = true)
#savefig("decision tree 2")

In [180]:

p = boxplot(train_acc_overall,label = "train accuracy", c = 3)
p1 = boxplot(test_acc_overall,label = "test accuracy",
    title = "overall accuracy",legend = :topleft,c = 6);

plot(p,p1)
#savefig("decision tree 3")

In [181]:
density(train_acc_overall,label = "train")
density!(test_acc_overall,label ="test",
    title = "overall accuracy",legend = :topleft)
#savefig("decision tree 4")

### random forest

In [182]:
model_1 = RandomForestClassifier(n_trees = 500,impurity_importance = true)
DecisionTree.fit!(model_1,training_pred,train_response)

RandomForestClassifier
n_trees:             500
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [0, 1]
ensemble:            Ensemble of Decision Trees
Trees:      500
Avg Leaves: 184.348
Avg Depth:  18.74

In [183]:
DecisionTree.impurity_importance(model_1)

10-element Vector{Float64}:
 0.029230803608716147
 0.30198515204907367
 0.029836558024260655
 0.027673238377459747
 0.023896948240944505
 0.049448262290976326
 0.02866674167300748
 0.2364927071931682
 0.21000542306183875
 0.06276416548055405

In [184]:
prediction_train = DecisionTree.predict(model_1,Matrix(train_pred))
acc_train = mean(prediction_train .== train_response)

0.9955935004131093

In [185]:
train_conf_mat = DecisionTree.confusion_matrix(train_response,prediction_train).matrix

2×2 Matrix{Int64}:
 3453    0
   16  162

In [186]:
countmap(train_response)

Dict{Int64, Int64} with 2 entries:
  0 => 3453
  1 => 178

In [187]:
prediction_test = DecisionTree.predict(model_1,Matrix(test_pred));
acc_test = mean(prediction_test .== test_response) # testing accuracy

0.9519621109607578

In [188]:
test_conf_mat = DecisionTree.confusion_matrix(test_response,prediction_test)

2×2 Matrix{Int64}:
 1407  0
   71  0

Classes:  [0, 1]
Matrix:   
Accuracy: 0.9519621109607578
Kappa:    0.0

In [189]:
countmap(test_response)

Dict{Int64, Int64} with 2 entries:
  0 => 1407
  1 => 71

## run the random forest model 500  times

In [ ]:
rep = 500
conf_mat_train = []
conf_mat_test = []
train_acc_overall = []
train_acc_correct = []
test_acc_overall = []
test_acc_correct = []
@time for i in 1:rep
    train,test = TrainTestSplit(new_data,0.7)
    train_pred = train[:,1:end-1]
    train_response = train[:,end];
    test_pred = test[:,1:end-1]
    test_response = test[:,end];
    training_pred = Matrix(train_pred);
    model = RandomForestClassifier(n_trees = 500,impurity_importance = true)
    DecisionTree.fit!(model,training_pred,train_response)
    prediction_train = DecisionTree.predict(model,Matrix(train_pred))
    push!(conf_mat_train,DecisionTree.confusion_matrix(train_response,prediction_train).matrix)
    push!(train_acc_overall,mean(prediction_train .== train_response))
    
    
    prediction_test = DecisionTree.predict(model,Matrix(test_pred));
    push!(conf_mat_test,DecisionTree.confusion_matrix(test_response,prediction_test).matrix)
    push!(test_acc_overall,mean(prediction_test .== test_response))
end

for i in 1:rep
    mat = conf_mat_train[i]
    push!(train_acc_correct,mat[2,2]/(mat[2,1] + mat[2,2]))
end

for i in 1:rep
    mat = conf_mat_test[i]
    push!(test_acc_correct,mat[2,2]/(mat[2,1] + mat[2,2]))
end

In [ ]:
median(train_acc_overall)

In [ ]:
median(test_acc_overall)

In [ ]:
median(train_acc_correct)

In [ ]:
mean(test_acc_correct)

In [ ]:
p = boxplot(train_acc_overall,label = "train accuracy", c = 3)
p1 = boxplot(test_acc_overall,label = "test accuracy",
    title = "overall accuracy ",legend = :topleft,c = 6);

plot(p,p1)

In [ ]:
density(train_acc_overall)
density!(test_acc_overall)

In [ ]:
p = boxplot(train_acc_correct,label = "train accuracy", c = 3)
p1 = boxplot(test_acc_correct,label = "test accuracy",
    title = "overall accuracy",legend = :topleft,c = 6);

plot(p,p1)

In [ ]:
countmap(new_data[:,end])

### Oversampling

In [ ]:
major_ind = findall(x -> x ==0,new_data[:,end])
minor_ind = findall(x -> x ==1,new_data[:,end]);

In [ ]:
smote(new_data[:,end-1],new_data[:,end])

In [ ]:
length(major_ind)

In [ ]:
length(minor_ind)

In [ ]:
minor = sample(minor_ind,length(major_ind));
resampled_data =vcat(new_data[minor,1:end],new_data[major_ind,1:end]);

In [ ]:
countmap(resampled_data[:,end])

In [ ]:
countmap(new_data[:,end])

In [ ]:
#major_ind = findall(x -> x ==0,new_data[:,end])
#minor_ind = findall(x -> x == 1,new_data[:,end])
#new_data[:,end] = vcat(sample(new_data[:,end][major_ind],4860),sample(new_data[:,end][minor_ind],4860));
train_data1,test_data1 = TrainTestSplit(resampled_data,0.7)
fm = @formula(stroke ~ age + hypertension + heart_disease+avg_glucose_level)
rev_log_model = glm(fm,train_data1,Binomial(),ProbitLink())
fit_predict1 = predict(rev_log_model,select(train_data1,Not([11])))
fit_predict1 = Vector{Int64}(fit_predict1 .> cut_off)
v1 = roc(fit_predict1,train_data1[:,end])
conf_mat1 = [v1.tn v1.fp; v1.fn v1.tp]
acc_train = ((conf_mat1[2,2] + conf_mat1[1,1])/sum(conf_mat1))*100 # this is training accuracy

rev_log_model = glm(fm,test_data1,Binomial(),ProbitLink())
fit_predict1 = predict(rev_log_model,select(test_data1,Not([11])))
fit_predict1 = Vector{Int64}(fit_predict1 .> cut_off)
v1 = roc(fit_predict1,test_data1[:,end])
conf_mat1 = [v1.tn v1.fp; v1.fn v1.tp]
acc_test = ((conf_mat1[2,2]+conf_mat1[1,1])/sum(conf_mat1))*100 # calculate the test accuracy

In [ ]:
acc_train,acc_test

In [ ]:

train,test = TrainTestSplit(resampled_data,0.7) 
# that means 70% data is used for building the model
train_pred = train[:,1:end-1]
train_response = train[:,end];
test_pred = test[:,1:end-1]
test_response = test[:,end]
rev_model_decision_tree = DecisionTreeClassifier(impurity_importance = true,
    max_depth = 7) 
    
# impurity importance is helps us to calculate the feature importance 
DecisionTree.fit!(rev_model_decision_tree,Matrix(train_pred),train_response)
train_prediction = DecisionTree.predict(rev_model_decision_tree,Matrix(train_pred))
acc_train = mean(train_prediction .== train_response)*100 #training accuracy
println(acc_train)
train_conf_mat = (DecisionTree.confusion_matrix(train_response,train_prediction)).matrix #confusion matrix for training data set
train_conf_mat


DecisionTree.fit!(rev_model_decision_tree,Matrix(test_pred),test_response)
prediction_test = DecisionTree.predict(rev_model_decision_tree,Matrix(test_pred));
acc_test = mean(prediction_test .== test_response)*100 
# test accuracy of the  decision tree model
println(acc_test)
test_conf_mat = (DecisionTree.confusion_matrix(test_response,prediction_test)).matrix #confusion matrix for training data set
#println(test_conf_mat)

In [ ]:
train_conf_mat

In [ ]:
train_conf_mat

In [ ]:
countmap(train_response)

In [ ]:
countmap(test_response)

In [ ]:
countmap(new_data[:,end])

In [ ]:
major_ind = findall(x -> x ==0,new_data[:,end])
minor_ind = findall(x -> x == 1,new_data[:,end]);

In [ ]:
new_data[:,end] = vcat(sample(new_data[:,end][major_ind],2600),sample(new_data[:,end][minor_ind],2509));

In [ ]:
countmap(new_data[:,end])

In [ ]:
train,test = TrainTestSplit(new_data,0.7) # that means 70% data is used for building the model
train_pred = train[:,1:end-1]
train_response = train[:,end];
test_pred = test[:,1:end-1]
test_response = test[:,end];

In [ ]:
size(train),size(test)

In [ ]:
countmap(train_response)

In [ ]:
countmap(test_response)

In [ ]:
training_pred = Matrix(train_pred);

In [ ]:
rev_model_decision_tree = DecisionTreeClassifier(impurity_importance = true,max_depth = 7,n_subfeatures = 3,min_samples_leaf = 5,
    min_samples_split = 6,pruning_purity_threshold = 4)# max_depth = 7,
# impurity importance is helps us to calculate the feature importance 
DecisionTree.fit!(rev_model_decision_tree,training_pred,train_response)

In [ ]:
print_tree(rev_model_decision_tree)

In [ ]:
prediction_train = DecisionTree.predict(rev_model_decision_tree,training_pred)
acc_train = mean(prediction_train .== train_response)*100

In [ ]:
train_conf_mat = (DecisionTree.confusion_matrix(train_response,
        prediction_train))

In [ ]:
countmap(train_response)

In [ ]:
prediction_test = DecisionTree.predict(rev_model_decision_tree,Matrix(test_pred));
acc_test = mean(prediction_test .== test_response)*100 
# test accuracy of the  decision tree model

In [ ]:
test_conf_mat = DecisionTree.confusion_matrix(test_response,prediction_test)

## random forest after over and under sampling

In [ ]:
rev_random_forest = RandomForestClassifier(n_trees = 500,impurity_importance = true,max_depth = 10)
DecisionTree.fit!(rev_random_forest,training_pred,train_response)

In [ ]:
prediction_train = DecisionTree.predict(rev_random_forest,Matrix(train_pred))
acc_train = mean(prediction_train .== train_response)*100

In [ ]:
train_conf_mat = DecisionTree.confusion_matrix(train_response,prediction_train)

In [ ]:
prediction_test = DecisionTree.predict(rev_random_forest,Matrix(test_pred));
acc_test = mean(prediction_test .== test_response)*100 # testing accuracy

In [ ]:
test_conf_mat = DecisionTree.confusion_matrix(test_response,prediction_test)

In [ ]:
countmap(test_response)

In [ ]:
countmap(train_response)

In [ ]:

new_data[:,end] = vcat(sample(new_data[:,end][major_ind],2550),sample(new_data[:,end][minor_ind],2559));

In [ ]:
countmap(new_data[:,end])

In [ ]:
major_ind = findall(x -> x ==0,new_data[:,end])
minor_ind = findall(x -> x == 1,new_data[:,end])
new_data[:,end] = vcat(sample(new_data[:,end][major_ind],2600),sample(new_data[:,end][minor_ind],2509));
train,test = TrainTestSplit(new_data,0.7) 
# that means 70% data is used for building the model
train_pred = train[:,1:end-1]
train_response = train[:,end];
test_pred = test[:,1:end-1]
test_response = test[:,end]
rev_model_decision_tree = RandomForestClassifier(impurity_importance = true,
    max_depth = 7) 
    
# impurity importance is helps us to calculate the feature importance 
DecisionTree.fit!(rev_model_decision_tree,Matrix(train_pred),train_response)
train_prediction = DecisionTree.predict(rev_model_decision_tree,Matrix(train_pred))
acc_train = mean(train_prediction .== train_response)*100 #training accuracy
println(acc_train)
train_conf_mat = (DecisionTree.confusion_matrix(train_response,train_prediction)).matrix #confusion matrix for training data set
train_conf_mat


DecisionTree.fit!(rev_model_decision_tree,Matrix(test_pred),test_response)
prediction_test = DecisionTree.predict(rev_model_decision_tree,Matrix(test_pred));
acc_test = mean(prediction_test .== test_response)*100 
# test accuracy of the  decision tree model
println(acc_test)
test_conf_mat = (DecisionTree.confusion_matrix(test_response,prediction_test)).matrix #confusion matrix for training data set
#println(test_conf_mat)

In [ ]:
train_conf_mat

In [ ]:
countmap(train_response)

In [ ]:
countmap(test_response)

In [ ]:
using Random

In [ ]:
shuffle!((1:nrow(data)) .% 10)